# Skip-gram

In [1]:
import numpy as np
from collections import defaultdict
import random

# ---------------------
# 1. Prepare the corpus
# ---------------------
corpus = [
    "king is a strong man",
    "queen is a wise woman",
    "man and woman are humans",
    "prince is a young king",
    "princess is a young queen"
]

# Tokenize each sentence into a list of lowercase words
tokenized_corpus = [sentence.lower().split() for sentence in corpus]

# Build word frequency dictionary
word_counts = defaultdict(int)
for sentence in tokenized_corpus:
    for word in sentence:
        word_counts[word] += 1

# Create vocabulary and lookup tables
vocab = list(word_counts.keys())
word2idx = {w: idx for idx, w in enumerate(vocab)}    # word to index mapping
idx2word = {idx: w for w, idx in word2idx.items()}    # index to word mapping
vocab_size = len(vocab)

# ----------------------------
# 2. Generate skip-gram pairs
# ----------------------------
def generate_training_data(tokenized_corpus, window_size=2):
    """
    For each word in a sentence, return (center, context) pairs within the given window.
    """
    pairs = []
    for sentence in tokenized_corpus:
        for idx, center_word in enumerate(sentence):
            for j in range(-window_size, window_size + 1):
                context_idx = idx + j
                if j != 0 and 0 <= context_idx < len(sentence):
                    context_word = sentence[context_idx]
                    pairs.append((center_word, context_word))
    return pairs

# Create training data
training_data = generate_training_data(tokenized_corpus)

# ----------------------------
# 3. Initialize model weights
# ----------------------------
embedding_dim = 10   # Size of word embedding vector

# Weight matrix from input to hidden layer
W1 = np.random.rand(vocab_size, embedding_dim)

# Weight matrix from hidden to output layer
W2 = np.random.rand(embedding_dim, vocab_size)

# -----------------------
# 4. Softmax and training
# -----------------------
def softmax(x):
    """
    Stable softmax function to convert raw scores into probabilities.
    """
    e_x = np.exp(x - np.max(x))  # stability trick
    return e_x / e_x.sum(axis=0)

def train(epochs=1000, learning_rate=0.01):
    """
    Train the model using naive softmax cross-entropy loss and SGD.
    """
    global W1, W2
    for epoch in range(epochs):
        loss = 0
        for center, context in training_data:
            # Create one-hot vector for center word
            x = np.zeros(vocab_size)
            x[word2idx[center]] = 1

            # Forward pass
            h = np.dot(W1.T, x)          # hidden layer (center word projection)
            u = np.dot(W2.T, h)          # output layer scores
            y_pred = softmax(u)          # predicted probability distribution

            # True distribution (one-hot for context word)
            y_true = np.zeros(vocab_size)
            y_true[word2idx[context]] = 1

            # Compute error
            e = y_pred - y_true

            # Backpropagate errors to weights
            dW2 = np.outer(h, e)
            dW1 = np.outer(x, np.dot(W2, e))

            # Update weights
            W1 -= learning_rate * dW1
            W2 -= learning_rate * dW2

            # Accumulate loss
            loss += -np.log(y_pred[word2idx[context]])
        
        # Print loss every 100 epochs
        if epoch % 100 == 0:
            print(f"Epoch {epoch}, Loss: {loss:.4f}")

# Train the model
train()

# -------------------
# 5. Query word vector
# -------------------
def get_word_vector(word):
    """
    Return the learned embedding vector for a given word.
    """
    idx = word2idx[word]
    return W1[idx]

# Show learned vectors for "king" and "queen"
print("\nVector for 'king':")
print(get_word_vector("king"))

print("\nVector for 'queen':")
print(get_word_vector("queen"))

Epoch 0, Loss: 204.4869
Epoch 100, Loss: 119.8568
Epoch 200, Loss: 112.9452
Epoch 300, Loss: 112.0204
Epoch 400, Loss: 111.7854
Epoch 500, Loss: 111.6963
Epoch 600, Loss: 111.6511
Epoch 700, Loss: 111.6222
Epoch 800, Loss: 111.6004
Epoch 900, Loss: 111.5823

Vector for 'king':
[ 0.94666437  0.73740089  0.78218087 -0.13031644 -0.51329574  1.50188143
  0.39904126  1.57611522 -0.05634789  0.66174988]

Vector for 'queen':
[ 1.18255719  0.56217685  0.95310334  0.22998969 -0.38948249  1.09408709
  0.153157    1.99713324 -0.10473538  1.34576386]


# Embeddings with transformer models
- BERT (Bidirectional Encoder Representations from Transformers): embeddings for word not sentence. BERT by itself wasn’t designed to produce a single vector for a whole sentence comparison out-of-the-box
- S-BERT

In [3]:
%pip install sentence_transformers

  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached huggingface_hub-0.34.3-py3-none-any.whl.metadata (14 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached requests-2.32.4-py3-none-any.whl.metadata (4.9 kB)
  Using cached tokenizers-0.21.4-cp39-abi3-macosx_11_0_arm64.whl.metadata (6.7 kB)
  Using cached fsspec-2025.7.0-py3-none-any.whl.metadata (12 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 63.8 MB/s  0:00:00
Using cached huggingface_hub-0.34.3-py3-none-any.whl (558 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 80.7 MB/s  0:00:00
Using cached tokenizers-0.21.4-cp39-abi3-macosx_11_0_arm64.whl (

In [4]:
# Sentence BERT (S-BERT)

from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')
sentence = "The quick brown fox jumps over the lazy dog."
embedding = model.encode(sentence)
print(embedding.shape)  # (384,)

/Users/alifouladgar/Documents/llm-useful/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/alifouladgar/Documents/llm-useful/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


(384,)
